# Ticket 6: Desarrollo de una Red Neuronal Básica

En este notebook se construye y entrena una red neuronal simple (MLP) utilizando PyTorch, con el objetivo de clasificar estados de atención (alta vs baja) a partir de características extraídas de la señal EEG. La red toma como input las features calculadas (energía en bandas delta, theta, alpha y beta) y se entrena para predecir la etiqueta. Se evaluará el desempeño y se comparará con el modelo clásico previo.

In [1]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print('Librerías importadas correctamente.')

Librerías importadas correctamente.


In [2]:
# Cargar el archivo v1p.mat y asignar nombres de canales
from scipy.io import loadmat

mat_data = loadmat('v1p.mat', squeeze_me=True)
data_array = mat_data['v1p']

df = pd.DataFrame(data_array)
channel_names = ['Fz', 'Cz', 'Pz', 'C3', 'T3', 'C4', 'T4', 'Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'T5', 'T6', 'O1', 'O2']
if df.shape[1] == len(channel_names):
    df.columns = channel_names
else:
    print('Advertencia: El número de columnas en el dataset no coincide con el número de canales esperados.')

print('Datos EEG cargados. Forma:', df.shape)

Datos EEG cargados. Forma: (12258, 19)


In [3]:
# --- Segmentación y Extracción de Features ---

# Parámetros de segmentación
fs = 128  # Frecuencia de muestreo (Hz)
segment_length = fs  # 1 segundo = 128 muestras
num_samples = df.shape[0]
num_segments = num_samples // segment_length
print('Número de segmentos:', num_segments)

# Extraer la señal del canal Fz
signal = df['Fz'].values

# Dividir la señal en segmentos
segments = []
for i in range(num_segments):
    start = i * segment_length
    end = start + segment_length
    segments.append(signal[start:end])
segments = np.array(segments)
print('Forma de la matriz de segmentos:', segments.shape)

# Función para calcular la potencia en una banda de frecuencia
from scipy.signal import welch

def bandpower(data, fs, band, nperseg=128):
    """Calcula la potencia de la señal en una banda específica usando Welch."""
    f, Pxx = welch(data, fs=fs, nperseg=nperseg)
    freq_idx = (f >= band[0]) & (f <= band[1])
    return np.trapz(Pxx[freq_idx], f[freq_idx])

# Definir bandas de frecuencia
bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30)
}

# Extraer features para cada segmento (energía en cada banda)
features_list = []
for seg in segments:
    features = {}
    for band_name, band_range in bands.items():
        features[band_name] = bandpower(seg, fs, band_range, nperseg=segment_length)
    features_list.append(features)

features_df = pd.DataFrame(features_list)
print('Primeros 5 registros de features extraídos:')
print(features_df.head())

Número de segmentos: 95
Forma de la matriz de segmentos: (95, 128)


C:\Users\Luna\AppData\Local\Temp\ipykernel_8300\2925495082.py:29: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(Pxx[freq_idx], f[freq_idx])


Primeros 5 registros de features extraídos:
           delta         theta        alpha         beta
0   32459.691406  16640.219482  4972.186279  6119.483767
1  245707.286133   9474.197876  3153.559708  4741.840615
2    7549.710449   3950.868713   908.005569  1339.171967
3    2155.812531   3338.061127  2545.681030  1929.724113
4   25218.478149  14299.832520  2690.533386  1543.227493


In [4]:
# --- Asignación de Etiquetas Simuladas ---

# Se simulan etiquetas basadas en la potencia en banda alpha
median_alpha = features_df['alpha'].median()
print('Valor mediano de potencia en alpha:', median_alpha)

# Asignar etiqueta: 1 para alta atención (Control), 0 para baja atención (TDAH)
features_df['Label'] = (features_df['alpha'] > median_alpha).astype(int)
features_df['Label_str'] = features_df['Label'].map({1: 'Control', 0: 'TDAH'})

print('Tabla de features con etiquetas:')
print(features_df.head())

Valor mediano de potencia en alpha: 2406.608139038086
Tabla de features con etiquetas:
           delta         theta        alpha         beta  Label Label_str
0   32459.691406  16640.219482  4972.186279  6119.483767      1   Control
1  245707.286133   9474.197876  3153.559708  4741.840615      1   Control
2    7549.710449   3950.868713   908.005569  1339.171967      0      TDAH
3    2155.812531   3338.061127  2545.681030  1929.724113      1   Control
4   25218.478149  14299.832520  2690.533386  1543.227493      1   Control


In [5]:
# --- Preparación de los Datos para PyTorch ---

# Seleccionar features y etiquetas
X = features_df[['delta', 'theta', 'alpha', 'beta']].values.astype(np.float32)
y = features_df['Label'].values.astype(np.int64)

# Dividir en entrenamiento y prueba (70% train, 30% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('Tamaño entrenamiento:', X_train.shape, '| Tamaño prueba:', X_test.shape)

# Convertir a tensores
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

# Crear DataLoader para batches
from torch.utils.data import TensorDataset, DataLoader
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Tamaño entrenamiento: (66, 4) | Tamaño prueba: (29, 4)


In [6]:
# --- Definición de la Arquitectura de la Red Neuronal (MLP) ---

class MLP(nn.Module):
    def __init__(self, input_dim=4, hidden_dim1=16, hidden_dim2=8, output_dim=2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Instanciar el modelo
model = MLP()
print('Arquitectura del modelo:')
print(model)

Arquitectura del modelo:
MLP(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=8, out_features=2, bias=True)
)


In [7]:
# --- Definir Función de Costo, Optimizador y Ciclo de Entrenamiento ---

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    acc = accuracy_score(all_targets, all_preds)
    print("Precisión en test:", acc)
    print("Matriz de confusión:")
    print(confusion_matrix(all_targets, all_preds))
    print("Reporte de clasificación:")
    print(classification_report(all_targets, all_preds))

print('Entrenando la red neuronal...')
train_model(model, train_loader, criterion, optimizer, num_epochs=50)

print('\nEvaluando el modelo en datos de prueba...')
evaluate_model(model, test_loader)

Entrenando la red neuronal...
Epoch 1/50, Loss: 329.6317
Epoch 2/50, Loss: 146.8162
Epoch 3/50, Loss: 70.8063
Epoch 4/50, Loss: 9.0916
Epoch 5/50, Loss: 8.2279
Epoch 6/50, Loss: 1.8707
Epoch 7/50, Loss: 1.1260
Epoch 8/50, Loss: 1.0364
Epoch 9/50, Loss: 0.6134
Epoch 10/50, Loss: 0.6794
Epoch 11/50, Loss: 0.6794
Epoch 12/50, Loss: 0.6764
Epoch 13/50, Loss: 0.6372
Epoch 14/50, Loss: 0.6319
Epoch 15/50, Loss: 0.6244
Epoch 16/50, Loss: 0.6212
Epoch 17/50, Loss: 0.6202
Epoch 18/50, Loss: 0.6196
Epoch 19/50, Loss: 0.5688
Epoch 20/50, Loss: 0.5968
Epoch 21/50, Loss: 0.5957
Epoch 22/50, Loss: 0.5562
Epoch 23/50, Loss: 0.5696
Epoch 24/50, Loss: 0.4900
Epoch 25/50, Loss: 0.5352
Epoch 26/50, Loss: 0.5761
Epoch 27/50, Loss: 0.5747
Epoch 28/50, Loss: 0.5257
Epoch 29/50, Loss: 0.6033
Epoch 30/50, Loss: 0.5708
Epoch 31/50, Loss: 0.5699
Epoch 32/50, Loss: 0.6041
Epoch 33/50, Loss: 0.6046
Epoch 34/50, Loss: 0.5691
Epoch 35/50, Loss: 0.6409
Epoch 36/50, Loss: 0.6040
Epoch 37/50, Loss: 0.6035
Epoch 38/50,

## Conclusiones e Interpretación

El modelo de red neuronal MLP fue entrenado utilizando las características extraídas de segmentos de EEG (potencia en bandas delta, theta, alpha y beta). Se entrenó y evaluó en un conjunto de datos simulados para clasificar estados de atención (alta vs baja). La precisión, la matriz de confusión y el reporte de clasificación permiten valorar el desempeño del modelo. Además, el análisis de los coeficientes (o la importancia de cada feature) puede sugerir qué bandas de frecuencia son más relevantes para distinguir los estados mentales.

**Consejo Profesional:**

El desarrollo de una red neuronal requiere una correcta estructuración de los datos, la selección adecuada de la arquitectura y el ajuste meticuloso de hiperparámetros. Experimenta con diferentes configuraciones, técnicas de regularización y estrategias de validación para mejorar la robustez y precisión del modelo. Una evaluación cuidadosa de las features y su relevancia es clave para avanzar en proyectos de inteligencia artificial.